In [9]:
import torch 
import matplotlib.pyplot as plt
import random
import torch.nn as nn

In [ ]:
# --------------Torchifying MLP yayy---------------

In [37]:
# Similiar api to that of pytorch

In [36]:
class Linear:                  # torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)

    def __init__ (self, fan_in, fan_out, biases=True):              # fan in-out tells the number of inputs and outputs
        self.weights = torch.randn((fan_in, fan_out), generator=g)  # Complete fan in out later
        self.bias = torch.zeros(fan_out) if bias else None
                                                               
    def __call__(self, X):
        if self.biases:
            self.h = X @ self.weights + self.biases 
        else:
            self.h = X @ self.weights
        return self.h
    
    def parameters(self):
            return sum(self.weights, ([] if self.bias is None else [self.bias]))

class BatchNorm1d:
# BatchNorm1d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)

    def __init__(self, fan_in, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        self.gamma = torch.ones(fan_in)
        self.beta = torch.zeros(fan_in)
    def __call__(self, X, Y):
        mean = X.mean(0, keepdim=True)
        var = X.var(0, keepdim=True)
#         y = (x - mean) / sqrt var[x] + epsilon  * gamma + beta
        Xhat = (X - mean) / torch.sqrt(var + self.eps) * self.gamma + self.beta # Xhat will represent the normalized version of the input
    
    def parameters(self):
        return [self.gamma, self.beta]

class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    def parameters(self):
        return []

g = torch.Generator().manual_seed(2147483647)               
n_emb = 10                                     # Will represent the dim
n_hidden = 100                                 # Represents the hidden layer neurons

